In [1]:
from bs4 import BeautifulSoup
import html
import re

In [3]:
def read_data(data_path):
    with open(data_path, 'r') as file:
        raw = file.read()
        
    pattern_reu = re.compile(r'<REUTERS .*>([\s\S]+?)<\/REUTERS>')
    pattern_title = re.compile(r'<TITLE>([\s\S]+?)<\/TITLE>')
    pattern_body = re.compile(r'<BODY>([\s\S]+?)<\/BODY>')
    pattern_unproc = re.compile(r'<TEXT TYPE="UNPROC">')
    pattern_brief = re.compile(r'<TEXT TYPE="BRIEF">')
    
    reuters = pattern_reu.findall(raw)
    
    articles = []
    for reu in reuters:
        if pattern_unproc.search(reu) is None and pattern_brief.search(reu) is None:
    #         m = pattern_title.search(reu) 
    #         title = m.group(1)
    #         print(title)
    #         print(html.unescape(title))

            m = pattern_body.search(reu) 
            body = m.group(1)
            body = html.unescape(body).strip().strip('Reuter')
        
            body = body.replace('\n',' ')
            
            articles.append(body)
        
    return articles
    


In [4]:
def read_all_data(data_dir):
    file_ids = list(range(0,22))
    data_paths = list(map(lambda x: '{}/reut2-{:03d}.sgm'.format(data_dir, x), file_ids))
#     print(data_paths)

    all_data = []
    for data_path in data_paths:
    #     print(data_path)
        articles = read_data(data_path)
        all_data.extend(articles)
        
    return all_data

In [5]:
data_path = 'data/reuters/reut2-000.sgm'
articles = read_data(data_path)
# print(content.replace('\n','')[:10000])

print(len(articles))
print(articles[0])

925
Showers continued throughout the week in the Bahia cocoa zone, alleviating the drought since early January and improving prospects for the coming temporao, although normal humidity levels have not been restored, Comissaria Smith said in its weekly review.     The dry period means the temporao will be late this year.     Arrivals for the week ended February 22 were 155,221 bags of 60 kilos making a cumulative total for the season of 5.93 mln against 5.81 at the same stage last year. Again it seems that cocoa delivered earlier on consignment was included in the arrivals figures.     Comissaria Smith said there is still some doubt as to how much old crop cocoa is still available as harvesting has practically come to an end. With total Bahia crop estimates around 6.4 mln bags and sales standing at almost 6.2 mln there are a few hundred thousand bags still in the hands of farmers, middlemen, exporters and processors.     There are doubts as to how much of this cocoa would be fit for exp

In [6]:
data_dir = 'data/reuters'
all_data = read_all_data(data_dir)
print(len(all_data))
print(all_data[:5])

19043
['Showers continued throughout the week in the Bahia cocoa zone, alleviating the drought since early January and improving prospects for the coming temporao, although normal humidity levels have not been restored, Comissaria Smith said in its weekly review.     The dry period means the temporao will be late this year.     Arrivals for the week ended February 22 were 155,221 bags of 60 kilos making a cumulative total for the season of 5.93 mln against 5.81 at the same stage last year. Again it seems that cocoa delivered earlier on consignment was included in the arrivals figures.     Comissaria Smith said there is still some doubt as to how much old crop cocoa is still available as harvesting has practically come to an end. With total Bahia crop estimates around 6.4 mln bags and sales standing at almost 6.2 mln there are a few hundred thousand bags still in the hands of farmers, middlemen, exporters and processors.     There are doubts as to how much of this cocoa would be fit for

In [ ]:
from nltk.tokenize import sent_tokenize

In [ ]:
def seg_sentences(text):
    sents = sent_tokenize(text)
    return sents

In [ ]:
test = content[:10000]

In [ ]:
# sents = sent_tokenize(test)
sents = seg_sentences(test)
print(len(sents))
print(sents[-5:])

In [35]:
import sklearn
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
import pandas as pd

In [9]:
vectorizer = CountVectorizer(min_df = 1, stop_words = 'english')

In [10]:
dtm = vectorizer.fit_transform(all_data)

In [11]:
pd.DataFrame(dtm.toarray(),columns=vectorizer.get_feature_names()).head(10)

,00,000,0000,00000,0002,0009,000s,001,0010,0013,...,zur,zurack,zurich,zurn,zuyuan,zverev,zweig,zwermann,zy,üside
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
dtm = dtm.asfptype()
type(dtm)

scipy.sparse.csr.csr_matrix

In [26]:
lsa = TruncatedSVD(100, algorithm = 'arpack')
dtm_lsa = lsa.fit_transform(dtm)


In [27]:
print(type(dtm_lsa))
dtm_lsa = Normalizer(copy=False).fit_transform(dtm_lsa)

<class 'numpy.ndarray'>


In [28]:
pd.DataFrame(lsa.components_, columns =vectorizer.get_feature_names())


,00,000,0000,00000,0002,0009,000s,001,0010,0013,...,zur,zurack,zurich,zurn,zuyuan,zverev,zweig,zwermann,zy,üside
0,0.005584,0.110666,1.701044e-04,0.000011,9.029718e-06,1.138793e-05,2.390599e-06,0.000202,2.938784e-06,7.154204e-06,...,0.000007,0.000030,0.000589,1.276749e-06,2.439649e-05,0.000012,3.180278e-06,0.000024,6.345176e-06,0.000028
1,0.001919,0.259585,3.808897e-04,0.000052,-2.894529e-06,6.008333e-05,1.513707e-07,0.000642,-3.513571e-06,-4.968709e-06,...,-0.000008,-0.000030,-0.000506,-1.608431e-06,-1.982266e-05,-0.000015,-2.619744e-07,-0.000030,-1.554139e-06,-0.000034
2,0.015496,-0.251251,-4.771509e-04,-0.000035,-3.077056e-06,-4.095776e-05,1.627023e-05,0.000555,-8.200954e-06,-8.773529e-06,...,-0.000021,-0.000062,-0.000505,-3.616567e-06,-2.559763e-05,-0.000032,5.194787e-06,-0.000021,-2.606296e-06,-0.000057
3,0.015433,0.133380,2.004054e-04,0.000009,-7.563114e-07,1.535101e-05,1.801543e-05,0.002628,3.871184e-06,2.407791e-06,...,0.000010,0.000044,0.000292,1.738383e-06,2.544488e-05,0.000018,-4.194937e-06,0.000013,-2.887110e-06,0.000028
4,-0.002581,-0.311287,-4.615263e-04,-0.000037,6.677134e-06,-4.321172e-05,-3.475190e-06,-0.000474,3.839467e-06,5.208493e-06,...,0.000017,0.000009,-0.000491,1.062885e-06,-9.306048e-06,0.000006,3.561837e-06,-0.000066,2.066562e-05,-0.000019
5,-0.006570,0.189859,2.958836e-04,0.000023,9.262241e-07,3.011179e-05,-5.558377e-06,0.001036,-1.943866e-06,-1.161394e-06,...,-0.000004,-0.000047,-0.000316,-1.207255e-06,-2.954555e-05,-0.000010,1.006367e-05,-0.000018,2.526270e-05,0.000017
6,0.002795,0.311750,2.840488e-04,0.000015,-2.621589e-05,2.927847e-06,-1.149413e-06,0.000682,3.053170e-06,-2.409157e-05,...,0.000022,-0.000012,-0.000786,6.158649e-07,-3.017261e-05,-0.000007,1.573965e-05,-0.000069,2.121281e-05,-0.000085
7,0.024798,0.223137,9.515828e-05,0.000011,-1.383592e-05,4.921373e-06,4.010138e-05,-0.002056,1.364375e-07,-1.177666e-05,...,-0.000010,0.000018,-0.000557,-3.222893e-07,3.265679e-05,0.000005,1.104177e-06,-0.000037,7.883042e-06,-0.000013
8,0.021967,-0.067803,2.135166e-04,-0.000001,5.931492e-05,2.487984e-05,-6.619241e-06,-0.000292,-5.301874e-06,4.864780e-05,...,0.000005,0.000038,0.002598,-2.241788e-06,-9.265880e-05,-0.000029,1.368390e-05,0.000101,8.360239e-06,0.000019
9,-0.002663,-0.368247,-8.939571e-05,-0.000032,-5.365159e-05,3.322968e-05,-1.429835e-05,-0.000084,1.389538e-05,-3.600000e-05,...,0.000062,0.000052,-0.000647,3.971894e-06,-1.319461e-05,0.000015,-7.126265e-06,-0.000057,-1.600995e-05,-0.000014


In [29]:
term2ind = {term:i for i, term in enumerate(vectorizer.get_feature_names())}

In [42]:
car = lsa.components_[:,term2ind['car']].reshape(1, -1)
cat = lsa.components_[:,term2ind['cat']].reshape(1, -1)
automobile = lsa.components_[:,term2ind['automobile']].reshape(1, -1)
# print(car)
# print(automobile)
print(cosine_similarity(car, automobile))
print(cosine_similarity(cat, automobile))


[[0.69949634]]
[[0.25568311]]
